In [1]:
from PyPDF2 import PdfReader
from dotenv import load_dotenv
import os
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pymilvus import MilvusClient

from tqdm.auto import tqdm
import numpy as np


/home/vscode/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
API_KEY=os.getenv('OPEN_API_KEY')
zilli_api_key=os.getenv('zilliAPI_KEY')


In [3]:
def extract_text_with_pypdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

In [4]:
# pdf_file_path = "../data/ifrs_doc.pdf"
pdf_file_path = "../data/ifrs_800pgs.pdf"

In [5]:
text = extract_text_with_pypdf(pdf_file_path)

In [6]:
client = OpenAI(api_key=API_KEY)


In [7]:
## chunking texts:

chunk_size = 6000 #openai model limit is 8191
chunk_overlap = 300

text_splitter = RecursiveCharacterTextSplitter(
                                                chunk_size=chunk_size,
                                                chunk_overlap=chunk_overlap
                                            )
chunks = text_splitter.split_text(text)

In [8]:
# embedding the chunks;
def emb_text(text):
    return (
        client.embeddings.create(input=text, model="text-embedding-3-small")
        .data[0]
        .embedding
    )

In [9]:

embeddings = []

for chunk in tqdm(chunks, total=len(chunks)):
    embeddings.append(emb_text(chunk))

  0%|          | 0/350 [00:00<?, ?it/s]

100%|██████████| 350/350 [02:20<00:00,  2.49it/s]


In [14]:
len(embeddings[0])

1536

In [15]:
embedding_dim = len(embeddings[0])

In [11]:
milvus_client = MilvusClient(uri="https://in03-cdcb64273091f72.serverless.gcp-us-west1.cloud.zilliz.com",
                             token=zilli_api_key)

collection_name = "demo_rag_collection"


In [35]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection, connections, utility


In [33]:
milvus_client = connections.connect(uri="https://in03-cdcb64273091f72.serverless.gcp-us-west1.cloud.zilliz.com",
                             token=zilli_api_key)

collection_name = "demo_rag_collection"


In [36]:
if collection_name in utility.list_collections():
    # Drop the collection if it exists
    utility.drop_collection(collection_name)
    print(f"Collection '{collection_name}' has been dropped.")

Collection 'demo_rag_collection' has been dropped.


In [37]:
# if milvus_client.has_collection(collection_name):
#     milvus_client.drop_collection(collection_name)


fields = [
        FieldSchema(name='id', dtype=DataType.INT64, is_primary=True),
        FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, dim=1536),
        FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=8000)
]

schema = CollectionSchema(fields, description="Embedding Collection for SmartAudit")

collection = Collection(collection_name, schema=schema)

In [38]:
# inserting embeddings to collection
ids = list(range(len(embeddings)))  # Auto ID list
vectors_to_insert = np.array(embeddings, dtype=np.float32)
texts = [chunk for chunk in chunks]  

In [29]:
texts[0]

'IFRS 1\nFirst-time Adoption of International\nFinancial Reporting Standards\nIn April 2001 the International Accounting Standards Board (Board) adopted SIC-8  First-\ntime Application of IASs as the Primary Basis of Accounting , which had been issued by the\nStanding Interpretations Committee of the International Accounting Standards\nCommittee in July 1998.\nIn June 2003 the Board issued IFRS 1 First-time Adoption of International Financial Reporting\nStandards  to replace SIC -8. IAS 1 Presentation of Financial Statements  (as revised in 2007)\namended the terminology used throughout IFRS Standards, including IFRS 1.\nThe Board restructured IFRS 1 in November 2008. In December 2010 the Board amended\nIFRS 1 to reflect that a first-time adopter would restate past transactions from the date of\ntransition to IFRS Standards instead of at 1 January 2004.\nSince it was issued in 2003, IFRS 1 was amended to accommodate first-time adoption\nrequirements resulting from new or amended Standa

In [39]:
collection.insert([ids, vectors_to_insert, texts])


(insert count: 350, delete count: 0, upsert count: 0, timestamp: 453254575556395011, success count: 350, err count: 0, cost: 1036)

In [40]:
# Define index parameters
index_params = {
    "index_type": "IVF_FLAT",  # Or "IVF_SQ8", "HNSW" depending on your needs
    "metric_type": "L2",  # Use L2 for Euclidean distance
    "params": {"nlist": 128}  # Number of clusters
}

# Create the index on the embedding field
collection.create_index(field_name="embedding", index_params=index_params)


Status(code=0, message=)

In [41]:
# Load the collection to make it available for queries
collection.load()


In [16]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

milvus_client.create_collection(
    collection_name=collection_name,
    dimension=1536,
    metric_type="IP",  # Inner product distance
    consistency_level="Strong",  # Strong consistency level
)


In [24]:
# inserting embeddings to collection
ids = list(range(len(embeddings)))  # Auto ID list
vectors_to_insert = np.array(embeddings, dtype=np.float32)  

In [25]:
vectors_to_insert.shape

(350, 1536)

In [19]:
insert_data = [
    vectors_to_insert  # Milvus accepts this as the embeddings data.
]

In [ ]:
data = []

for i, line in enumerate(tqdm(vectors_to_insert, desc="Creating embeddings")):
    data.append({"id": i, "vector": emb_text(line), "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

In [27]:
data = []
for i, chunk in enumerate(tqdm(chunks, total=len(chunks))):
    data.append({"id": i, "vector":emb_text(chunk), "text":chunk})

100%|██████████| 350/350 [02:33<00:00,  2.29it/s]


In [30]:
milvus_client.insert(collection_name=collection_name, data=data)


{'insert_count': 350, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

In [41]:
index_params = MilvusClient.prepare_index_params()

index_params.add_index(
    field_name="vector",
    metric_type="COSINE",
    index_type="IVF_FLAT",
    index_name="vector_index",
    params={ "nlist": 128 }
)

milvus_client.create_index(
    collection_name="demo_rag_collection",
    index_params=index_params,
    sync=False # Whether to wait for index creation to complete before returning. Defaults to True.
)



RPC error: [create_index], <MilvusException: (code=65535, message=CreateIndex failed: creating multiple indexes on same field is not supported)>, <Time:{'RPC start': '2024-10-15 20:15:04.254534', 'RPC error': '2024-10-15 20:15:25.368749'}>
Failed to create an index on collection: demo_rag_collection


MilvusException: <MilvusException: (code=65535, message=CreateIndex failed: creating multiple indexes on same field is not supported)>

In [43]:
milvus_client.drop_index(collection_name)


TypeError: drop_index() missing 1 required positional argument: 'index_name'

In [46]:
res = milvus_client.list_indexes(
    collection_name=collection_name
)

print(res)

['vector']


In [31]:
# Building Retrieval ;

In [42]:
question = "what is the IFRS standard for new companies looking to audit their records?"


In [43]:
results = collection.search(
    data=[
        emb_text(question)
    ],
    anns_field="embedding",
    param={"nprobe": 10},  # Number of clusters to search
    limit=5,
    metric_type="L2"
)


In [45]:
results

data: ["['id: 1, distance: 0.913716197013855, entity: {}', 'id: 4, distance: 0.9649375081062317, entity: {}', 'id: 2, distance: 0.9797342419624329, entity: {}', 'id: 0, distance: 0.9857110381126404, entity: {}', 'id: 209, distance: 0.9971702694892883, entity: {}']"], cost: 6

In [48]:
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
results = collection.search([emb_text(question)], "embedding", param=search_params, limit=5, output_fields=["text"])


In [49]:
results

data: ['["id: 1, distance: 0.9135565161705017, entity: {\'text\': \'Standards\\\\nObjective\\\\nThe objective of this IFRS is to ensure that an entity’s first IFRS financial\\\\nstatements , and its interim financial reports for part of the period covered by\\\\nthose financial statements, contain high quality information that:\\\\n(a) is transparent for users and comparable over all periods presented;\\\\n(b) provides a suitable starting point for accounting in accordance\\\\nwith International Financial Reporting Standards (IFRSs) ; and\\\\n(c) can be generated at a cost that does not exceed the benefits.\\\\nScope\\\\nAn entity shall apply this IFRS in:\\\\n(a) its first IFRS financial statements; and\\\\n(b) each interim financial report, if any, that it presents in accordance\\\\nwith IAS 34 Interim Financial Reporting  for part of the period covered by\\\\nits first IFRS financial statements.\\\\nAn entity’s first IFRS financial statements are the first annual financial\\\\nstate

In [57]:
results

data: ['["id: 1, distance: 0.9135565161705017, entity: {\'text\': \'Standards\\\\nObjective\\\\nThe objective of this IFRS is to ensure that an entity’s first IFRS financial\\\\nstatements , and its interim financial reports for part of the period covered by\\\\nthose financial statements, contain high quality information that:\\\\n(a) is transparent for users and comparable over all periods presented;\\\\n(b) provides a suitable starting point for accounting in accordance\\\\nwith International Financial Reporting Standards (IFRSs) ; and\\\\n(c) can be generated at a cost that does not exceed the benefits.\\\\nScope\\\\nAn entity shall apply this IFRS in:\\\\n(a) its first IFRS financial statements; and\\\\n(b) each interim financial report, if any, that it presents in accordance\\\\nwith IAS 34 Interim Financial Reporting  for part of the period covered by\\\\nits first IFRS financial statements.\\\\nAn entity’s first IFRS financial statements are the first annual financial\\\\nstate

In [67]:
results[0][0][0]

TypeError: 'Hit' object is not subscriptable

In [64]:
dict(results[0][0])
     

MilvusException: <MilvusException: (code=1, message=Field keys is not in the hit entity)>

In [60]:
for res in results:
    print(res)

["id: 1, distance: 0.9135565161705017, entity: {'text': 'Standards\\nObjective\\nThe objective of this IFRS is to ensure that an entity’s first IFRS financial\\nstatements , and its interim financial reports for part of the period covered by\\nthose financial statements, contain high quality information that:\\n(a) is transparent for users and comparable over all periods presented;\\n(b) provides a suitable starting point for accounting in accordance\\nwith International Financial Reporting Standards (IFRSs) ; and\\n(c) can be generated at a cost that does not exceed the benefits.\\nScope\\nAn entity shall apply this IFRS in:\\n(a) its first IFRS financial statements; and\\n(b) each interim financial report, if any, that it presents in accordance\\nwith IAS 34 Interim Financial Reporting  for part of the period covered by\\nits first IFRS financial statements.\\nAn entity’s first IFRS financial statements are the first annual financial\\nstatements in which the entity adopts IFRSs , by

In [59]:
# Assuming 'results' is your SearchResult object
for hit in results:
    # Each hit is an instance of Hit
    id = hit.id
    distance = hit.distance
    entity = hit.entity  # This contains your additional fields

    # Assuming 'text' is a field within the entity
    text = entity.get("text", "No text found")  # Safely retrieve text

    # Print or store the retrieved information
    print(f"ID: {id}, Distance: {distance}, Text: {text}")


AttributeError: 'Hits' object has no attribute 'id'

In [44]:
import json
retrieved_lines_with_distances = [
    (res.entity.text, res.distance) for res in results[0]
]

# Print the formatted results as JSON
print(json.dumps(retrieved_lines_with_distances, indent=4))


MilvusException: <MilvusException: (code=1, message=Field text is not in the hit entity)>

In [47]:
search_res = collection.search(
    collection_name=collection_name,
    data=[
        emb_text(question)
    ],  # Use the `emb_text` function to convert the question to an embedding vector
    anns_field="embedding",
    param={"nprobe": 10}, 
    limit=3,  # Return top 3 results
    search_params={"metric_type": "IP", "params": {}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)


Unexpected error: [search], search() got multiple values for argument 'collection_name', <Time: {'RPC start': '2024-10-15 21:54:18.084918', 'Exception': '2024-10-15 21:54:18.084947'}>


MilvusException: <MilvusException: (code=1, message=Unexpected error, message=<search() got multiple values for argument 'collection_name'>)>

In [33]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=[
        emb_text(question)
    ],  # Use the `emb_text` function to convert the question to an embedding vector
    limit=3,  # Return top 3 results
    search_params={"metric_type": "IP", "params": {}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)


In [26]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in results[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))


TypeError: 'Hit' object is not subscriptable

In [35]:
## Use LLM to get a RAG response

In [36]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)


In [74]:
for res in results[0]:
    print(res)

id: 1, distance: 0.9135565161705017, entity: {'text': 'Standards\nObjective\nThe objective of this IFRS is to ensure that an entity’s first IFRS financial\nstatements , and its interim financial reports for part of the period covered by\nthose financial statements, contain high quality information that:\n(a) is transparent for users and comparable over all periods presented;\n(b) provides a suitable starting point for accounting in accordance\nwith International Financial Reporting Standards (IFRSs) ; and\n(c) can be generated at a cost that does not exceed the benefits.\nScope\nAn entity shall apply this IFRS in:\n(a) its first IFRS financial statements; and\n(b) each interim financial report, if any, that it presents in accordance\nwith IAS 34 Interim Financial Reporting  for part of the period covered by\nits first IFRS financial statements.\nAn entity’s first IFRS financial statements are the first annual financial\nstatements in which the entity adopts IFRSs , by an explicit and u

In [75]:
len

TypeError: sequence item 0: expected str instance, Hit found

In [37]:
SYSTEM_PROMPT = """
Human: You are an AI assistant. You are able to find answers to the questions from the contextual passage snippets provided.
"""
USER_PROMPT = f"""
Use the following pieces of information enclosed in <context> tags to provide an answer to the question enclosed in <question> tags.
<context>
{context}
</context>
<question>
{question}
</question>
"""


In [38]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)


In [39]:
print(response.choices[0].message.content)


The relevant IFRS standard for new companies looking to audit their records is IFRS 1, which provides guidelines for entities preparing their first IFRS financial statements. IFRS 1 outlines the requirements and procedures for entities adopting International Financial Reporting Standards (IFRS) for the first time, ensuring that their financial statements contain high-quality information that is transparent, comparable, and generated at a reasonable cost.
